In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-seller-order-status-prediction/orders_data.xlsx


Observing the dataset

In [2]:
pip install openpyxl

     |████████████████████████████████| 242 kB 249 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
myfile = pd.read_excel("../input/amazon-seller-order-status-prediction/orders_data.xlsx")
myfile.head(10)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,"CHANDIGARH,",CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,₹449.00,NaN,NaN,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,₹449.00,₹60.18,NaN,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,"DEVARAKONDA,",TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,NaN,NaN,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,"MUMBAI,",MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",₹84.96,NaN,Delivered to buyer
5,406-9292208-6725123,"Thu, 17 Jun, 2021, 9:12 pm IST",Subinita,"HOWRAH,",WEST BENGAL,SKU: HH-FOWV-5YWO,Women's Trendy Pure Leather Clutch Purse | Lea...,1,₹200.00,NaN,NaN,Delivered to buyer
6,404-5794317-7737924,"Thu, 12 Aug, 2021, 8:03 pm IST",shailendra,"ORAI,",UTTAR PRADESH,SKU: TQ-OE6K-9DIK,Ultra Slim 100% Pure Leather Men's Wallet with...,1,NaN,NaN,Cash On Delivery,Returned to seller
7,405-8702211-4054722,"Wed, 29 Sept, 2021, 2:55 pm IST",Pratima,"BAREILLY,",UTTAR PRADESH,SKU: S1-A92Q-JU3X,100% Pure Leather Shantiniketan Clutch Purse: ...,1,₹399.00,₹84.96,Cash On Delivery,Delivered to buyer
8,171-1434812-8061163,"Sat, 13 Nov, 2021, 7:37 pm IST",Ipshita,"BENGALURU,",KARNATAKA,SKU: 3F-4R9N-Z8NJ,Set of 2 Pure Leather Block Print Round Jewelr...,1,₹399.00,₹84.96,NaN,Delivered to buyer
9,171-7954707-4463549,"Mon, 9 Aug, 2021, 4:47 pm IST",A.Jayaprada,"Bhilai,",CHHATTISGARH,SKU: NU-CKZ5-4O49,Pure Leather Sling Bag with Multiple Pockets a...,1,"₹1,099.00",NaN,NaN,Delivered to buyer


In [4]:
#Data Treatment

#check empty
myfile.isnull().sum()
#does cod coincide with NaN shipping fees \ cod>shipping fees


order_no          0
order_date        0
buyer             0
ship_city         0
ship_state        0
sku               0
description       0
quantity          0
item_total       18
shipping_fee     26
cod             124
order_status      0
dtype: int64

In [5]:
myfile.order_status.unique()

array(['Delivered to buyer', 'Returned to seller'], dtype=object)

In [6]:
# imputing with mode as shipping fee is fixed based on package size and weight
myfile['shipping_fee'].fillna(myfile['shipping_fee'].mode()[0], inplace=True)

# imputing with mode as we are considering the item that is sold the most
myfile['item_total'].fillna(myfile['item_total'].mode()[0], inplace=True)

# assuming if an order is not cod would mean the mode of payment is online
myfile['cod'].fillna('online', inplace=True)

#Because there are missing values in features {item_total, shipping_fee and cod}, we can input them with mode of the features, as shipping fee is fixed based on package size and weight.

#fillna() replaces NULL values with specified value
#if inplace=True, replacing done on current DataFrame

for i, value in enumerate(myfile["item_total"]):
    value = value.translate({ord("₹"):None})
    value = value.translate({ord(","):None})
    myfile["item_total"][i]=value


for i, value in enumerate(myfile["shipping_fee"]):
    value = value.translate({ord("₹"):None})
    value = value.translate({ord(","):None})
    myfile["shipping_fee"][i]=value

myfile.head()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,"CHANDIGARH,",CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.00,84.96,online,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,"DEVARAKONDA,",TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,449.00,84.96,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,"MUMBAI,",MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,1099.00,84.96,online,Delivered to buyer


In [7]:
myfile.isnull().sum()


order_no        0
order_date      0
buyer           0
ship_city       0
ship_state      0
sku             0
description     0
quantity        0
item_total      0
shipping_fee    0
cod             0
order_status    0
dtype: int64

In [8]:
myfile.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   order_no      171 non-null    object
 1   order_date    171 non-null    object
 2   buyer         171 non-null    object
 3   ship_city     171 non-null    object
 4   ship_state    171 non-null    object
 5   sku           171 non-null    object
 6   description   171 non-null    object
 7   quantity      171 non-null    int64 
 8   item_total    171 non-null    object
 9   shipping_fee  171 non-null    object
 10  cod           171 non-null    object
 11  order_status  171 non-null    object
dtypes: int64(1), object(11)
memory usage: 16.2+ KB


In [9]:
myfile.head(5)
#len(myfile.buyer.unique())
#d = 149/171
#print(d)

,order_no,order_date,buyer,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status
0,405-9763961-5211537,"Sun, 18 Jul, 2021, 10:38 pm IST",Mr.,"CHANDIGARH,",CHANDIGARH,SKU: 2X-3C0F-KNJE,100% Leather Elephant Shaped Piggy Coin Bank |...,1,449.00,84.96,online,Delivered to buyer
1,404-3964908-7850720,"Tue, 19 Oct, 2021, 6:05 pm IST",Minam,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
2,171-8103182-4289117,"Sun, 28 Nov, 2021, 10:20 pm IST",yatipertin,"PASIGHAT,",ARUNACHAL PRADESH,SKU: DN-0WDX-VYOT,Women's Set of 5 Multicolor Pure Leather Singl...,1,449.00,60.18,online,Delivered to buyer
3,405-3171677-9557154,"Wed, 28 Jul, 2021, 4:06 am IST",aciya,"DEVARAKONDA,",TELANGANA,SKU: AH-J3AO-R7DN,Pure 100% Leather Block Print Rectangular Jewe...,1,449.00,84.96,Cash On Delivery,Delivered to buyer
4,402-8910771-1215552,"Tue, 28 Sept, 2021, 2:50 pm IST",Susmita,"MUMBAI,",MAHARASHTRA,SKU: KL-7WAA-Z82I,Pure Leather Sling Bag with Multiple Pockets a...,1,1099.00,84.96,online,Delivered to buyer


In [10]:
#Data Processing
from sklearn import preprocessing
import numpy as np

#drop order_status, order no, buyer
#encoder for sku, description, ship_state, ship_city, quantity
#separate order date into day of week / month:day / time
#binarizer cod 


def feature_eng(df, to_encode, to_del):
    
    lb = preprocessing.LabelBinarizer()
    df["cod"] = lb.fit_transform(df["cod"])
    df["order_status"] = lb.fit_transform(df["cod"])

    #splitting order_date to dd:mm:yy time
    df['order_date'] = df['order_date'].astype(str)
    df[['day','date','year', 'time']]=df["order_date"].str.split(",",expand=True) 
    #'yy' is used to catch empty space 
    df[['yy','dd', 'mm']]=myfile["date"].str.split(" ",expand=True)
    df['ship_city'] = myfile['ship_city'].apply(lambda x: x.replace(',', ''))
    
    le = preprocessing.LabelEncoder()
    for i in to_encode:
        df[i] = le.fit_transform(df[i])
    
    df['quantity'] = df['quantity'].astype(float)
    df['item_total'] = df['item_total'].astype(float)
    df['shipping_fee'] = df['shipping_fee'].astype(float)

    df.drop(axis=1,columns=to_del, inplace=True)

    return df
#has approximately 90% unique buyers, so repeat chances of order status is not further investigated
to_del = ["order_no", "buyer","order_date", "year", "yy", "date"]
to_encode = ["ship_city", "ship_state","sku", "description", "day", "dd", "mm"]
feature_eng(myfile, to_encode, to_del)

#need to convert time to 24 hour system

,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status,day,time,dd,mm
0,13,5,2,1,1.0,449.0,84.96,1,1,3,10:38 pm IST,9,4
1,56,1,22,59,1.0,449.0,60.18,1,1,5,6:05 pm IST,10,7
2,56,1,22,59,1.0,449.0,60.18,1,1,3,10:20 pm IST,20,6
3,18,25,17,30,1.0,449.0,84.96,0,0,6,4:06 am IST,20,4
4,47,17,29,37,1.0,1099.0,84.96,1,1,5,2:50 pm IST,20,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,47,17,22,59,3.0,1347.0,84.96,0,0,1,11:30 am IST,4,1
167,17,28,42,52,1.0,1299.0,114.46,1,1,6,12:18 pm IST,0,1
168,19,7,42,52,1.0,1299.0,105.02,1,1,4,6:55 pm IST,30,1
169,39,29,30,50,1.0,1499.0,80.24,0,0,6,12:43 am IST,15,2


**Cleaning dataset: removing commas, checking for null and repeated entries**

In [11]:
myfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ship_city     171 non-null    int64  
 1   ship_state    171 non-null    int64  
 2   sku           171 non-null    int64  
 3   description   171 non-null    int64  
 4   quantity      171 non-null    float64
 5   item_total    171 non-null    float64
 6   shipping_fee  171 non-null    float64
 7   cod           171 non-null    int64  
 8   order_status  171 non-null    int64  
 9   day           171 non-null    int64  
 10  time          171 non-null    object 
 11  dd            171 non-null    int64  
 12  mm            171 non-null    int64  
dtypes: float64(3), int64(9), object(1)
memory usage: 17.5+ KB


In [12]:
myfile.head()

,ship_city,ship_state,sku,description,quantity,item_total,shipping_fee,cod,order_status,day,time,dd,mm
0,13,5,2,1,1.0,449.0,84.96,1,1,3,10:38 pm IST,9,4
1,56,1,22,59,1.0,449.0,60.18,1,1,5,6:05 pm IST,10,7
2,56,1,22,59,1.0,449.0,60.18,1,1,3,10:20 pm IST,20,6
3,18,25,17,30,1.0,449.0,84.96,0,0,6,4:06 am IST,20,4
4,47,17,29,37,1.0,1099.0,84.96,1,1,5,2:50 pm IST,20,8


# **Machine Learning Algorithm**

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


Y = myfile.order_status
X = myfile[["ship_city","ship_state","item_total","shipping_fee","cod","day","dd", "mm"]]
X_train, X_test, y_train, y_test = train_test_split(X ,Y, random_state=42)

tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Prediction of test set:{:.3f}".format(tree.score(X_test, y_test)))
print("Making predictions for the following 5 orders:")
print(X_test.head())
print("The predictions are")
print(tree.predict(X_test.head()))

Accuracy on training set: 1.000
Prediction of test set:1.000
Making predictions for the following 5 orders:
     ship_city  ship_state  item_total  shipping_fee  cod  day  dd  mm
101         51           8       549.0         84.96    0    4  19   3
55          39          29       399.0         47.20    1    0   6   7
56           4          17       175.0         84.96    1    6   7   5
139         42          29       549.0         47.20    1    2  25   1
157          8          10       399.0         84.96    0    6  29   1
The predictions are
[0 1 1 1 0]


In [14]:
from sklearn.tree import DecisionTreeRegressor

tree1 = DecisionTreeRegressor(random_state=1)
tree1.fit(X_train,y_train)
print("Making predictions for the following 5 vehicles:")
print(X.head())
print("The predictions are")
print(tree1.predict(X.head()))
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Prediction of test set:{:.3f}".format(tree.score(X_test, y_test)))

Making predictions for the following 5 vehicles:
   ship_city  ship_state  item_total  shipping_fee  cod  day  dd  mm
0         13           5       449.0         84.96    1    3   9   4
1         56           1       449.0         60.18    1    5  10   7
2         56           1       449.0         60.18    1    3  20   6
3         18          25       449.0         84.96    0    6  20   4
4         47          17      1099.0         84.96    1    5  20   8
The predictions are
[1. 1. 1. 0. 1.]
Accuracy on training set: 1.000
Prediction of test set:1.000


In [15]:
#overfitting occurs
#checking feature importance
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [16]:
fig = px.histogram(myfile, x="cod", y="item_total", color='order_status', barmode='group', height=400)
fig.show()

Most orders are made online and the percentage of return orders is more in cod mode.